# Modal

[Modal 云平台](https://modal.com/docs/guide)提供便捷的按需访问功能，让您能够从本地计算机上的 Python 脚本获得无服务器云计算能力。
使用 `modal` 来运行您自己的定制 LLM 模型，而不是依赖 LLM API。

本示例将介绍如何使用 LangChain 与 `modal` 的 HTTPS [Web 端点](https://modal.com/docs/guide/webhooks)进行交互。

[_使用 LangChain 进行问答_](https://modal.com/docs/guide/ex/potus_speech_qanda) 是另一个展示如何与 `Modal` 一同使用 LangChain 的示例。在该示例中，Modal 端到端地运行 LangChain 应用程序，并使用 OpenAI 作为其 LLM API。

In [ ]:
%pip install --upgrade --quiet  modal

In [2]:
# Register an account with Modal and get a new token.

!modal token new

Launching login page in your browser window...
If this is not showing up, please copy this URL into your web browser manually:
https://modal.com/token-flow/tf-Dzm3Y01234mqmm1234Vcu3


`langchain.llms.modal.Modal` 集成类要求您部署一个 Modal 应用，该应用需要有一个符合以下 JSON 接口的 Web 端点：

1. LLM 的提示作为键为 `"prompt"` 的 `str` 值接收。
2. LLM 的响应作为键为 `"prompt"` 的 `str` 值返回。

**示例请求 JSON：**

```json
{
    "prompt": "Identify yourself, bot!",
    "extra": "args are allowed",
}
```

**示例响应 JSON：**

```json
{
    "prompt": "This is the LLM speaking",
}
```

一个满足此接口的示例“虚拟”Modal Web 端点函数如下：

```python
...
...

class Request(BaseModel):
    prompt: str

@stub.function()
@modal.web_endpoint(method="POST")
def web(request: Request):
    _ = request  # ignore input
    return {"prompt": "hello world"}
```

* 有关设置符合此接口的端点的基础知识，请参阅 Modal 的 [Web 端点](https://modal.com/docs/guide/webhooks#passing-arguments-to-web-endpoints) 指南。
* 将 Modal 的 ['运行 Falcon-40B 配合 AutoGPTQ'](https://modal.com/docs/guide/ex/falcon_gptq) 开源 LLM 示例作为自定义 LLM 的起点！

一旦你部署了一个 Modal web endpoint，你就可以将它的 URL 传递给 `langchain.llms.modal.Modal` LLM 类。这个类随后就可以在你构建的链式结构中作为一个构建模块来使用。

In [ ]:
from langchain.chains import LLMChain
from langchain_community.llms import Modal
from langchain_core.prompts import PromptTemplate

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [ ]:
endpoint_url = "https://ecorp--custom-llm-endpoint.modal.run"  # REPLACE ME with your deployed Modal web endpoint's URL
llm = Modal(endpoint_url=endpoint_url)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)